In [2]:
library(mgcv)
library(dplyr)
library(gdata)
library(epitools)
library(tableone)

In [2238]:
head(df_mimic,2)

icustay_id,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,sofatotal,vasso_duration_24h,vasso_duration_48h,⋯,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h,bmi,AKI_7d,mortality_7d,use_vasopressor
271147,54,F,WHITE,SICU,0,0,2,13.8500000,13.8500000,⋯,47,50,50,66.5,68,68,31.65708,1,0,1
214619,21,M,HISPANIC,TSICU,0,0,2,0.9166667,0.9166667,⋯,67,72,72,84.0,91,92,25.42990,0,0,1


In [30]:
df<- read.csv("csv_folder/eicu_clean_19Aug.csv", stringsAsFactors = TRUE)
df$age<- as.integer(df$age)
df$gender<- factor(df$gender)
df$ethnicity<- factor(df$ethnicity)
cols <- c(18:29)
df[cols] <- lapply(df[cols], factor)  
df_eicu<- df[,-c(1,7,8,10,12,39,41)]
df_eicu$hosp_mort <-ifelse(df_eicu$hosp_mort=='ALIVE',0,1)
df_eicu$icu_mort<-ifelse(df_eicu$icu_mort=='ALIVE',0,1)
df_eicu$gender <-ifelse(df_eicu$gender=='Female','F','M')
#df_eicu$gender<-df_eicu$Gender
df_eicu <- df_eicu %>% rename(vasso_duration_24h = X24hrs_hr, 
                              vasso_duration_48h = X48hrs_hr,
                              vasso_duration_72h = X72hrs_hr,
                              mechanical_ventilation = ventilation_duration_days,
                              sepsis = sepsis_binary,
                              hospital_expire_flag = hosp_mort,
                              icu_mortality = icu_mort,
                              stroke = cebrovascular_accident,
                              afib = atrial_fibrillation
                                )
df_eicu$use_vasopressor<- ifelse(df_eicu$vasso_duration_24h>0,1,0)


df_eicu_aki <-df_eicu[df_eicu$mortality_7d == 'ALIVE',]
eicu_prop <- read.csv("csv_folder/eicu_prop_24h.csv",stringsAsFactors = TRUE)
df_eicu_prop <- merge(x = df_eicu, y = eicu_prop[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI <-df_eicu_prop
df_eicu_prop_AKI$prop_map_24_AKI<- 1- df_eicu_prop_AKI$propBelow_map_24
df_eicu_prop_AKI$prop_asp_24_AKI<- 1- df_eicu_prop_AKI$propBelow_asp_24
df_eicu_prop_AKI$prop_adp_24_AKI<- 1- df_eicu_prop_AKI$propBelow_adp_24
df_eicu_prop_AKI <-df_eicu_prop_AKI[df_eicu_prop_AKI$mortality_7d == 'ALIVE',]

eicu_prop_48 <- read.csv("csv_folder/eicu_prop_48h.csv",stringsAsFactors = TRUE)
df_eicu_prop_48 <- merge(x = df_eicu, y = eicu_prop_48[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI_48 <-df_eicu_prop_48
df_eicu_prop_AKI_48$prop_map_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_map_48
df_eicu_prop_AKI_48$prop_asp_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_asp_48
df_eicu_prop_AKI_48$prop_adp_48_AKI<- 1- df_eicu_prop_AKI_48$propBelow_adp_48
df_eicu_prop_AKI_48 <-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$mortality_7d == 'ALIVE',]

eicu_prop_72 <- read.csv("csv_folder/eicu_prop_72h.csv",stringsAsFactors = TRUE)
df_eicu_prop_72 <- merge(x = df_eicu, y = eicu_prop_72[ ,  -1], by = "patientunitstayid", all.x=TRUE)
df_eicu_prop_AKI_72 <-df_eicu_prop_72
df_eicu_prop_AKI_72$prop_map_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_map_72
df_eicu_prop_AKI_72$prop_asp_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_asp_72
df_eicu_prop_AKI_72$prop_adp_72_AKI<- 1- df_eicu_prop_AKI_72$propBelow_adp_72
df_eicu_prop_AKI_72 <-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$mortality_7d == 'ALIVE',]

head(df_eicu_prop_AKI, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24,prop_map_24_AKI,prop_asp_24_AKI,prop_adp_24_AKI
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,0.04166667,0.7500000,0.04166667,0.2083333,1.0000000,0.000000000,0.00000000,0.04166667,0.2500000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,0.50467290,0.3271028,0.45794393,0.2149533,0.9065421,0.009345794,0.08411215,0.65420561,0.6728972,0.09345794


In [31]:
df1 <- read.csv("csv_folder/mimic_clean_19Aug.csv", 
               stringsAsFactors = TRUE)
df1$age<- as.integer(df1$age)
df1$icu_mortality<- factor(df1$icu_mortality)

cols <- c(17:28,39,40 )
df1[cols] <- lapply(df1[cols], factor)  
df_mimic<- df1[,-c(1,7,8,10,12)]
df_mimic <- df_mimic %>% rename(vasso_duration_24h = first_24_vasso_hours, 
                                vasso_duration_48h = first_48_vasso_hours,
                                vasso_duration_72h = first_72_vasso_hours,
                                unittype = first_careunit,
                                ethnicity = ethnicity_new, 
                                AKI_7d =aki_7day,
                                sofatotal = sofa,
                                mechanical_ventilation = vent_duration_days,
                                chf = congestive_heart_failure,
                                stroke = cereb)
df_mimic$use_vasopressor <- ifelse(df_mimic$vasso_duration_24h>0,1,0)

df_mimic_aki <-df_mimic[df_mimic$mortality_7d == 0,]
mimic_prop <- read.csv("csv_folder/mimic_prop_24h.csv",stringsAsFactors = TRUE)
# mimic_prop_AKI <- read.csv("csv_folder/mimic_prop_map_48h_72h.csv",stringsAsFactors = TRUE)

df_mimic_prop <- merge(x = df_mimic, y = mimic_prop[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI <- df_mimic_prop
df_mimic_prop_AKI$prop_map_24_AKI<-1-df_mimic_prop_AKI$propBelow_map_24
df_mimic_prop_AKI$prop_asp_24_AKI<-1-df_mimic_prop_AKI$propBelow_asp_24
df_mimic_prop_AKI$prop_adp_24_AKI<-1-df_mimic_prop_AKI$propBelow_adp_24
df_mimic_prop_AKI <-df_mimic_prop_AKI[df_mimic_prop_AKI$mortality_7d==0,]

mimic_prop_48 <- read.csv("csv_folder/mimic_prop_48h.csv",stringsAsFactors = TRUE)
df_mimic_prop_48 <- merge(x = df_mimic, y = mimic_prop_48[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI_48 <-df_mimic_prop_48
df_mimic_prop_AKI_48$prop_map_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_map_48
df_mimic_prop_AKI_48$prop_asp_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_asp_48
df_mimic_prop_AKI_48$prop_adp_48_AKI<- 1- df_mimic_prop_AKI_48$propBelow_adp_48
df_mimic_prop_AKI_48 <-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$mortality_7d==0,]

mimic_prop_72 <- read.csv("csv_folder/mimic_prop_72h.csv",stringsAsFactors = TRUE)
df_mimic_prop_72 <- merge(x = df_mimic, y = mimic_prop_72[ ,  -1], by = "icustay_id", all.x=TRUE)
df_mimic_prop_AKI_72 <-df_mimic_prop_72
df_mimic_prop_AKI_72$prop_map_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_map_72
df_mimic_prop_AKI_72$prop_asp_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_asp_72
df_mimic_prop_AKI_72$prop_adp_72_AKI<- 1- df_mimic_prop_AKI_72$propBelow_adp_72
df_mimic_prop_AKI_72 <-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$mortality_7d==0,]

head(df_mimic_prop_AKI, n = 2)

icustay_id,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,sofatotal,vasso_duration_24h,vasso_duration_48h,⋯,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24,prop_map_24_AKI,prop_asp_24_AKI,prop_adp_24_AKI
200003,48,M,WHITE,SICU,0,0,6,1.084444,17.33444,⋯,0.4705882,0.4375000,0.1875000,0.3750000,0.06250000,0.7500000,0.1875000,0.9411765,0.5625000,0.9375000
200009,47,F,WHITE,CSRU,0,0,3,0.000000,0.00000,⋯,0.1935484,0.7096774,0.1612903,0.1290323,0.03225806,0.3548387,0.6129032,0.4516129,0.2903226,0.9677419


# table 1 eicu

In [584]:

xvars<-c("gender","age","bmi","ethnicity","unittype","sofatotal",
         "apache_iv",
         "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h",
#          "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#          "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h",
    
         "hypertension","afib","cancer","chf","ckd","cld","copd","diabetes","ihd","stroke",
         "sepsis",
         
         "hospital_expire_flag","AKI_7d","icu_mortality","vasso_duration_24h","use_vasopressor","mechanical_ventilation"
        
        )
catvars<-c("gender","ethnicity","unittype","hypertension","afib","cancer","chf","ckd","cld","copd","diabetes","ihd","stroke",
         "sepsis","hospital_expire_flag","AKI_7d","icu_mortality","use_vasopressor","mechanical_ventilation")

In [667]:

TableOne_eICU<-CreateTableOne(var = xvars, factorVars =catvars,data=df_eicu)

print(TableOne_eICU, nonnormal = c("age","bmi","sofatotal","apache_iv",
                                    "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h","vasso_duration_24h"
#                                     "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#                                     "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h"
                                   ), 
      contDigits = 1, catDigits = 1, pDigits = 4, smd = TRUE,testName=TRUE)


                                   
                                    Overall             
  n                                 14374               
  gender = M (%)                     8439 (58.7)        
  age (median [IQR])                 65.0 [55.0, 74.0]  
  bmi (median [IQR])                 28.3 [24.2, 33.0]  
  ethnicity (%)                                         
     African American                1827 (12.7)        
     Asian                            134 ( 0.9)        
     Caucasian                      11218 (78.0)        
     Hispanic                         501 ( 3.5)        
     Native American                  101 ( 0.7)        
     Others                           593 ( 4.1)        
  unittype (%)                                          
     Cardiac ICU                      526 ( 3.7)        
     CCU-CTICU                       1508 (10.5)        
     CSICU                           1000 ( 7.0)        
     CTICU                           1596 (11.1)    

# table1 mimic

In [668]:

TableOne_eICU<-CreateTableOne(var = xvars, factorVars =catvars,data=df_mimic)

print(TableOne_eICU, nonnormal = c("age","bmi","sofatotal",
                                   #"apache_iv",
                                    "ibp_systolic_24h", "ibp_diastolic_24h","ibp_mean_24h","vasso_duration_24h"
#                                     "ibp_systolic_48h", "ibp_diastolic_48h","ibp_mean_48h",
#                                     "ibp_systolic_72h","ibp_diastolic_72h","ibp_mean_72h"
                                   ), 
      contDigits = 1, catDigits = 1, pDigits = 4, smd = TRUE,testName=TRUE)


Warning message in ModuleReturnVarsExist(vars, data):
“The data frame does not have: apache_iv  Dropped”

                                   
                                    Overall             
  n                                 12291               
  gender = M (%)                     7228 (58.8)        
  age (median [IQR])                 66.0 [53.0, 76.0]  
  bmi (median [IQR])                 29.5 [25.7, 31.9]  
  ethnicity (%)                                         
     ASIAN                            291 ( 2.4)        
     BLACK                            735 ( 6.0)        
     HISPANIC                         366 ( 3.0)        
     OTHERS                          2354 (19.2)        
     WHITE                           8545 (69.5)        
  unittype (%)                                          
     CCU                             1290 (10.5)        
     CSRU                            3769 (30.7)        
     MICU                            2534 (20.6)        
     SICU                            2522 (20.5)        
     TSICU                           2176 (17.7)    

In [587]:
x<-df_eicu$vasso_duration_24h[df_eicu$vasso_duration_24h>0]
median(x)
quantile(x)

[1] 22.83

0%   25%   50%   75%  100% 
 0.02 17.43 22.83 23.95 24.00

In [5]:
# Model fitting
plot_width <- 17.2 * 0.393701 # The textwidth of our Word manuscript in inches.
#plot_width <- 4.2126 # The Lancet asks for plots to have a width of 107mm.
font <- "Times" # Text in figures should be Times New Roman according to the Lancet.
point_size <- 10 # Per Lancet instructions
logistic <- function(x) 1/(1+exp(-x))



# Function that adds labels to subfigures, edited from:
# https://logfc.wordpress.com/2017/03/15/adding-figure-labels-a-b-c-in-the-top-left-corner-of-the-plotting-region/
fig_label <- function(label) {
    cex <- 2
    ds <- dev.size("in")
    # xy coordinates of device corners in user coordinates
    x <- grconvertX(c(0, ds[1]), from="in", to="user")
    y <- grconvertY(c(0, ds[2]), from="in", to="user")
    
    # fragment of the device we use to plot
      # account for the fragment of the device that 
      # the figure is using
      fig <- par("fig")
      dx <- (x[2] - x[1])
      dy <- (y[2] - y[1])
      x <- x[1] + dx * fig[1:2]
      y <- y[1] + dy * fig[3:4]
  
  
  sw <- strwidth(label, cex=cex) * 60/100
  sh <- strheight(label, cex=cex) * 60/100
  
  x1 <- x[1] + sw
  
  y1 <- y[2] - sh
  
  old.par <- par(xpd=NA)
  on.exit(par(old.par))
  
  text(x1, y1, label, cex=cex)
}

In [6]:
gam_plotMed <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
# vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"
# colD <- "Blue"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
    eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
    ord.index <- order(xx)
    xx <- xx[ord.index]
    i<-2
    j<-1
    index <-1
    bp_val <-list()
    bp_grad<-list()
    current_list <-list()
    next_list<-list()
    while (i <=length(ord.index)){
        eval(parse(text = paste(c('x2 <- gamfitMed$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
        eval(parse(text = paste(c('x1 <- gamfitMed$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]

        if((x2-x1)>=1.0){
            current_val <- logistic(predictionsPlusCI$fit[ord.index[j]])
            right_val <- logistic(predictionsPlusCI$fit[ord.index[i]])
            grad <-(right_val-current_val)*100/(x2-x1)
            bp_val[[index]] <- x1
            bp_grad[[index]]<-unlist(grad,use.names=FALSE)
            current_list[[index]]<-current_val
            next_list[[index]]<-right_val
            index <-index+1
            j <- i
            i <- i+1

        } else {
            i <-i+1
        }

        }
    df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), 
                        "gradient" = unlist(bp_grad, use.names=FALSE)
                       , "pre_val_left" = unlist(current_list, use.names=FALSE)
                       , "pre_val_right" = unlist(next_list, use.names=FALSE))
    
    by_type <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
    if(startsWith(xName, 'ibp_mean', ignore.case=TRUE)){
        bp_value_left <-60
        bp_value_right<-90
    } else if (startsWith(xName, 'ibp_systolic', ignore.case=TRUE)){
        bp_value_left <-100
        bp_value_right<-140
        
    } else {
        bp_value_left <-40
        bp_value_right<-70
    }
    
    
    grad_0.2_left<-min(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_right<-max(df_gradient[(df_gradient$gradient > -0.2) & (df_gradient$gradient< 0.2)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_left<-min(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                                   & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.1_right<-max(df_gradient[(df_gradient$gradient > -0.1) & (df_gradient$gradient< 0.1)
                               & (df_gradient$bp_value< bp_value_right) & (df_gradient$bp_value> bp_value_left), "bp_value"])
    grad_0.2_left <-ifelse(grad_0.2_left>=43, grad_0.2_left,40)
    grad_0.1_left <-ifelse(grad_0.1_left>=43, grad_0.1_left,41)
    grad_0.2_right<-ifelse(grad_0.2_right<=137, grad_0.2_right,139)
    grad_0.1_right<-ifelse(grad_0.1_right<=137, grad_0.1_right,138)

    if(gamfitMed$family$link == 'logit') {
        lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
        ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])

        lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
        lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
        lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)
        abline(v=grad_0.2_left, col = "Red")
        abline(v=grad_0.2_right, col = "Red")
        abline(v=grad_0.1_left, col = "Blue")
        abline(v=grad_0.1_right, col = "Blue")
    } else {
        lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
        ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]

        lines(x = xx, y = lcl, lty = 2, lwd = 2)
        lines(x = xx, y = ucl, lty = 2, lwd = 2)
        lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
        abline(v=grad_0.2_left, col = "Red")
        abline(v=grad_0.2_right, col = "Red")
        abline(v=grad_0.1_left, col = "Blue")
        abline(v=grad_0.1_right, col = "Blue")
    }

    if(!missing(label)) fig_label(label)
}


In [7]:
find_gradient <- function(b)
{
xName <- colnames(b$model)[grep("ibp", colnames(b$model))]
eval(parse(text = paste(c('newdata = data.frame(',
                          xName, ' = b$model$', xName, ", gender = 'F', age = median(b$model$age), bmi = median(b$model$bmi),",
                  ifelse(
                    "high_vent_proportion" %in% colnames(b$model),
                    "high_vent_proportion = median(b$model$high_vent_proportion),",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"
                  ),
                  ifelse(
                    "ethnicity" %in% colnames(b$model),
                    "ethnicity = b$model$ethnicity,",
                    ""
                  ),
                  ifelse(
                    "apsiii" %in% colnames(b$model),
                    "apsiii = median(b$model$apsiii),",
                    "sofatotal = median(b$model$sofatotal),"

                  ),
                  ifelse(
                    "vasso_duration_24h" %in% colnames(b$model),
                    "vasso_duration_24h = median(b$model$vasso_duration_24h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_48h" %in% colnames(b$model),
                    "vasso_duration_48h = median(b$model$vasso_duration_48h),",
                    ""
                  ),
                  ifelse(
                    "vasso_duration_72h" %in% colnames(b$model),
                    "vasso_duration_72h = median(b$model$vasso_duration_72h),",
                    ""
                  ),
                  "hospital_id = 264)"), collapse = "")))
# newd <- newdata
pred<-predict(b,newdata,type="link",se.fit = T)
eval(parse(text = paste0('xx <- b$model$', xName)))
# xx <- b$model$ibp_mean_24h
ord.index <- order(xx)
i<-2
j<-1
index <-1
bp_val <-list()
bp_grad<-list()
current_list <-list()
next_list<-list()
while (i <=length(ord.index)){
    eval(parse(text = paste(c('x2 <- b$model$', xName,"[ord.index[",i,"]]"), collapse = "")))
    eval(parse(text = paste(c('x1 <- b$model$', xName,"[ord.index[",j,"]]"), collapse = "")))
#       x2 <- b$model$ibp_mean_24h[ord.index[i]]
#       x1 <-b$model$ibp_mean_24h[ord.index[j]]
    
    if((x2-x1)>=1.0){
        current_val <- logistic(pred$fit[ord.index[j]])
        right_val <- logistic(pred$fit[ord.index[i]])
        grad <-(right_val-current_val)*100/(x2-x1)
        bp_val[[index]] <- x1
        bp_grad[[index]]<-unlist(grad,use.names=FALSE)
        current_list[[index]]<-current_val
        next_list[[index]]<-right_val
        index <-index+1
        j <- i
        i <- i+1
        
      } else {
          i <-i+1
      }
      
    }
df_gradient <- data.frame("bp_value" = unlist(bp_val, use.names=FALSE), 
                          "gradient" = unlist(bp_grad, use.names=FALSE)
                         , "pre_val_left" = unlist(current_list, use.names=FALSE)
                         , "pre_val_right" = unlist(next_list, use.names=FALSE))
return (df_gradient)
}

In [8]:
gam_plotMed_noGradient <- function(
                    gamfitMed,
                    main = "Median of measurements",
                    xRange = c(55, 95),
                    yRange = c(0, .20),
                    label,
                    xlabel
) {
print(paste("Number of cases:", summary(gamfitMed)$n))

xlab <- xlabel


if(colnames(gamfitMed$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
    else if(colnames(gamfitMed$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  } 
    else {
    ylab <- "Probability of ICU mortality"
  }

xName <- colnames(gamfitMed$model)[grep("ibp", colnames(gamfitMed$model))]
vasso_Name <- colnames(gamfitMed$model)[grep("vasso", colnames(gamfitMed$model))]

# Color for dotted line
colD <- "Black"

    plot(1, type = 'n', xlim = xRange, ylim = yRange,
         ylab = ylab,
         xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
    
    att <- pretty(yRange)
if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
  axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
} else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
    
    att <- pretty(xRange)
    axis(1, at = att, lab = paste0(att), las = TRUE)
#     axis(1, at = att, lab = paste0(att, '%'), las = TRUE)

    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitMed, newdata = data.frame(',
                              xName, ' = gamfitMed$model$', xName, ", gender = 'F', age = median(gamfitMed$model$age), 
                            bmi = median(gamfitMed$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitMed$model),
                        "high_vent_proportion = median(gamfitMed$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitMed$model),
                        "ethnicity = gamfitMed$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitMed$model),
                        "apsiii = median(gamfitMed$model$apsiii),",
                        "sofatotal = median(gamfitMed$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitMed$model),
                        "vasso_duration_24h = median(gamfitMed$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitMed$model),
                        "vasso_duration_48h = median(gamfitMed$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitMed$model),
                        "vasso_duration_72h = median(gamfitMed$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))

  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors for 'new' input
  eval(parse(text = paste0('xx <- gamfitMed$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitMed$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = colD)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = colD)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2)
    lines(x = xx, y = ucl, lty = 2, lwd = 2)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3)
  }
  
  if(!missing(label)) fig_label(label)
}


## mean vs hospital mortality, ICU mortality and AKI

In [2241]:
# hospital mortality
gamfitMed_eICU_MAP_24h <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_24h <- gam(hospital_expire_flag ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_MAP_48h <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_48h <- gam(hospital_expire_flag ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_MAP_72h <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_MAP_72h <- gam(hospital_expire_flag ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)
# ICU mortality
gamfitMed_ICU_eICU_MAP_24h <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_24h <- gam(icu_mortality ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_MAP_48h <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_48h <- gam(icu_mortality ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_MAP_72h <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_MAP_72h <- gam(icu_mortality ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)
# AKI
gamfitMed_AKI_eICU_MAP_24h <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_24h <- gam(AKI_7d ~ s(ibp_mean_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_MAP_48h <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_48h <- gam(AKI_7d ~ s(ibp_mean_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_MAP_72h <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_MAP_72h <- gam(AKI_7d ~ s(ibp_mean_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [2242]:
pdf(file = "figure/gradient/MAP_Hosp_ICU_AKI.pdf", width = plot_width *3 ,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(55, 95)
gam_plotMed(gamfitMed_eICU_MAP_24h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "A1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MAP_48h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "C1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_eICU_MAP_72h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "E1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B1", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D1", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F1", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

x_range = c(55, 95)
gam_plotMed(gamfitMed_ICU_eICU_MAP_24h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "A2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_eICU_MAP_48h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "C2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_eICU_MAP_72h, main = "eICU", xRange = x_range,
                    yRange = c(0, .20), label = "E2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B2", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D2", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed(gamfitMed_ICU_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F2", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

x_range = c(50, 100)
gam_plotMed_noGradient(gamfitMed_AKI_eICU_MAP_24h, main = "eICU", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "A3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_MAP_48h, main = "eICU", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "C3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_MAP_72h, main = "eICU", xRange = x_range,
                    yRange = c(0.75, 0.95), label = "E3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_MAP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "B3", xlabel ="Median of 24h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_MAP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "D3", xlabel ="Median of 48h Mean Arterial Pressure (MAP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_MAP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0.6, 1.0),label = "F3", xlabel ="Median of 72h Mean Arterial Pressure (MAP)")
dev.off()

ERROR: Error in pdf(file = "figure/gradient/MAP_Hosp_ICU_AKI_7d.pdf", width = plot_width * : object 'plot_width' not found


## systolic vs hospital mortality, ICU mortality and AKI

In [15]:
# hospital
gamfitMed_eICU_ASP_24h <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_24h <- gam(hospital_expire_flag ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ASP_48h <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_48h <- gam(hospital_expire_flag ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ASP_72h <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ASP_72h <- gam(hospital_expire_flag ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

#ICU
gamfitMed_ICU_eICU_ASP_24h <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_24h <- gam(icu_mortality ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ASP_48h <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_48h <- gam(icu_mortality ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ASP_72h <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ASP_72h <- gam(icu_mortality ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)
# AKI

gamfitMed_AKI_eICU_ASP_24h <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_24h <- gam(AKI_7d ~ s(ibp_systolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ASP_48h <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_48h <- gam(AKI_7d ~ s(ibp_systolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ASP_72h <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ASP_72h <- gam(AKI_7d ~ s(ibp_systolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [19]:
pdf(file = "figure/gradient/ASP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
gam_plotMed(gamfitMed_eICU_ASP_24h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "A1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_ASP_48h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "C1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_eICU_ASP_72h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "E1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "B1", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "D1", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "F1", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_eICU_ASP_24h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "A2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_eICU_ASP_48h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "C2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_eICU_ASP_72h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0, .20), label = "E2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "B2", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "D2", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = c(0, .20),label = "F2", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

y_range = c(0.6, 1.0)
gam_plotMed_noGradient(gamfitMed_AKI_eICU_ASP_24h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "A3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_ASP_48h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "C3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_ASP_72h, main = "eICU", xRange = c(90, 140),
                    yRange = c(0.75, 0.95), label = "E3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ASP_24h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "B3", xlabel ="Median of 24h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ASP_48h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "D3", xlabel ="Median of 48h Systolic Blood Pressure (SBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ASP_72h, main = "MIMIC", xRange = c(90, 140),
                    yRange = y_range,label = "F3", xlabel ="Median of 72h Systolic Blood Pressure (SBP)")
dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"


png 
  2

## diastolic vs hospital, ICU and AKI

In [55]:
# hospital
gamfitMed_eICU_ADP_24h <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_24h <- gam(hospital_expire_flag ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ADP_48h <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_48h <- gam(hospital_expire_flag ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_eICU_ADP_72h <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_MIMIC_ADP_72h <- gam(hospital_expire_flag ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)


# ICU
gamfitMed_ICU_eICU_ADP_24h <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_24h <- gam(icu_mortality ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ADP_48h <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_48h <- gam(icu_mortality ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

gamfitMed_ICU_eICU_ADP_72h <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu, family = binomial)
gamfitMed_ICU_MIMIC_ADP_72h <- gam(icu_mortality ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic, family = binomial)

# AKI
gamfitMed_AKI_eICU_ADP_24h <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_24h <- gam(AKI_7d ~ s(ibp_diastolic_24h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_24h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ADP_48h <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_48h <- gam(AKI_7d ~ s(ibp_diastolic_48h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_48h), data = df_mimic_aki, family = binomial)

gamfitMed_AKI_eICU_ADP_72h <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_eicu_aki, family = binomial)
gamfitMed_AKI_MIMIC_ADP_72h <- gam(AKI_7d ~ s(ibp_diastolic_72h)+gender+s(age)+s(bmi)+s(sofatotal)+s(vasso_duration_72h), data = df_mimic_aki, family = binomial)


In [56]:
pdf(file = "figure/gradient/ADP_Hosp_ICU_AKI.pdf", width = plot_width*3,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,6), cex = 1)
x_range = c(35, 75)
gam_plotMed(gamfitMed_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B1", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D1", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F1", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")
x_range = c(35, 75)
gam_plotMed(gamfitMed_ICU_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "A2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_ICU_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "C2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_ICU_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = c(0, .20), label = "E2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "B2", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "D2", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed(gamfitMed_ICU_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = c(0, .20),label = "F2", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

x_range = c(35, 75)
y_range = c(0.75,0.95)
y_range2 = c(0.6,1.0)
gam_plotMed_noGradient(gamfitMed_AKI_eICU_ADP_24h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "A3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_ADP_48h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "C3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_AKI_eICU_ADP_72h, main = "eICU-CRD", xRange = x_range,
                    yRange = y_range, label = "E3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ADP_24h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "B3", xlabel ="Median of 24h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ADP_48h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "D3", xlabel ="Median of 48h Diastolic Blood Pressure (DBP)")

gam_plotMed_noGradient(gamfitMed_AKI_MIMIC_ADP_72h, main = "MIMIC", xRange = x_range,
                    yRange = y_range2,label = "F3", xlabel ="Median of 72h Diastolic Blood Pressure (DBP)")

dev.off()

[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 13105"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"
[1] "Number of cases: 11445"


png 
  2

# proportional plot function

In [9]:
gam_plotProp <- function(
  gamfitProp,
  main = "Effect of treatment regime",
  yRange = c(0, .2),
  add = FALSE, col = "black",
  label,
  bp_type = "MAP",
  bp_low = 70,
  bp_high = 80
){
  print(paste("Number of cases:", summary(gamfitProp)$n))
  
  xName <- colnames(gamfitProp$model)[grep("prop", colnames(gamfitProp$model))]
  
  xRange = c(min(gamfitProp$model[,xName], na.rm= TRUE), max(gamfitProp$model[,xName], na.rm= TRUE))
#   xRange = c(0, max(gamfitProp$model[,xName], na.rm= TRUE))
  
  if(startsWith(xName, 'propBelow', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type," <", bp_low, "mmHg (blue) or >",bp_high, "mmHg (red)")
    #xlab <- expression("Proportion of  SpO"[2]*" measurements below 94%")
  } else if(startsWith(xName, 'propAbove', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type, " above ", bp_high, "mmHg")
  } else {
    xlab <- paste("Proportion of ",bp_type, " within ",bp_low, "mmHg to ", bp_high, "mmHg")
  }
  
  if(colnames(gamfitProp$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
      else if(colnames(gamfitProp$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  }
      else {
    ylab <- "Probability of ICU mortality"
  }
  
  if(!add) {
    plot(1, type = 'n', xlim = xRange, ylim = yRange,
           ylab = ylab,
           xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
      
      att <- pretty(yRange)
    if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
      axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
    } else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
      
      att <- pretty(xRange)
      axis(1, at = att, lab = paste0(att*100,'%'), las = TRUE)
  
  }
    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitProp, newdata = data.frame(', xName, 
                              ' = gamfitProp$model$', xName, 
                              ", gender = 'F', age = median(gamfitProp$model$age), bmi = median(gamfitProp$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitProp$model),
                        "high_vent_proportion = median(gamfitProp$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitProp$model),
                        "ethnicity = gamfitProp$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitProp$model),
                        "apsiii = median(gamfitProp$model$apsiii),",
                        "sofatotal = median(gamfitProp$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitProp$model),
                        "vasso_duration_24h = median(gamfitProp$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitProp$model),
                        "vasso_duration_48h = median(gamfitProp$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitProp$model),
                        "vasso_duration_72h = median(gamfitProp$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))
    
    
  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors 
  # for 'new' input
  eval(parse(text = paste0('xx <- gamfitProp$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitProp$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3, col = col)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3, col = col)
  }
  
  if(!missing(label)) fig_label(label)
}

In [10]:
gam_plotProp_AKI <- function(
  gamfitProp,
  main = "Effect of treatment regime",
  yRange = c(0.6, 0.9),
  add = FALSE, col = "black",
  label,
  bp_type = "MAP",
  bp_low = 70
){
  print(paste("Number of cases:", summary(gamfitProp)$n))
  
  xName <- colnames(gamfitProp$model)[grep("prop", colnames(gamfitProp$model))]
  
  xRange = c(min(gamfitProp$model[,xName], na.rm= TRUE), max(gamfitProp$model[,xName], na.rm= TRUE))
#   xRange = c(0, max(gamfitProp$model[,xName], na.rm= TRUE))
  
  if(startsWith(xName, 'propBelow', ignore.case=TRUE)) {
    xlab <- paste("Proportion of ",bp_type," <", bp_low, "mmHg (blue)")
    #xlab <- expression("Proportion of  SpO"[2]*" measurements below 94%")
  }  else {
    xlab <- paste("Proportion of ",bp_type, " >= ",bp_low, "mmHg")
  }
  
  if(colnames(gamfitProp$model)[1] == "hospital_expire_flag") {
    ylab <- "Probability of hospital mortality"
  }
      else if(colnames(gamfitProp$model)[1] == "AKI_7d") {
    ylab <- "Probability of acute kidney injury (AKI)"
  }
      else {
    ylab <- "Probability of ICU mortality"
  }
  
  if(!add) {
    plot(1, type = 'n', xlim = xRange, ylim = yRange,
           ylab = ylab,
           xlab = xlab, main = main, yaxs = 'i', xaxs = 'i', yaxt = 'n', xaxt = 'n')
      
      att <- pretty(yRange)
    if(!isTRUE(all.equal(att, round(att, digits = 2)))) {
      axis(2, at = att, lab = paste0(sprintf('%.1f', att*100), '%'), las = TRUE)
    } else axis(2, at = att, lab = paste0(att*100, '%'), las = TRUE)
      
      att <- pretty(xRange)
      axis(1, at = att, lab = paste0(att*100,'%'), las = TRUE)
  
  }
    
    eval(parse(text = paste(c('predictionsPlusCI <- predict(gamfitProp, newdata = data.frame(', xName, 
                              ' = gamfitProp$model$', xName, 
                              ", gender = 'F', age = median(gamfitProp$model$age), bmi = median(gamfitProp$model$bmi),",
                      ifelse(
                        "high_vent_proportion" %in% colnames(gamfitProp$model),
                        "high_vent_proportion = median(gamfitProp$model$high_vent_proportion),",
                        ""
                      ),
                      ifelse(
                        "ethnicity" %in% colnames(gamfitProp$model),
                        "ethnicity = gamfitProp$model$ethnicity,",
                        ""
                      ),
                      ifelse(
                        "apsiii" %in% colnames(gamfitProp$model),
                        "apsiii = median(gamfitProp$model$apsiii),",
                        "sofatotal = median(gamfitProp$model$sofatotal),"
                        
                      ),
                      ifelse(
                        "vasso_duration_24h" %in% colnames(gamfitProp$model),
                        "vasso_duration_24h = median(gamfitProp$model$vasso_duration_24h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_48h" %in% colnames(gamfitProp$model),
                        "vasso_duration_48h = median(gamfitProp$model$vasso_duration_48h),",
                        ""
                      ),
                      ifelse(
                        "vasso_duration_72h" %in% colnames(gamfitProp$model),
                        "vasso_duration_72h = median(gamfitProp$model$vasso_duration_72h),",
                        ""
                      ),
                      "hospital_id = 264), type = 'link', se.fit = T)"), collapse = "")))
    
    
  
  # We have to use the data on which GAM was fit for confidence region as the GAM does not provide standard errors 
  # for 'new' input
  eval(parse(text = paste0('xx <- gamfitProp$model$', xName)))
  ord.index <- order(xx)
  xx <- xx[ord.index]
  
  if(gamfitProp$family$link == 'logit') {
    lcl <- logistic(predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index])
    ucl <- logistic(predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index])
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, logistic(predictionsPlusCI$fit[ord.index]), lwd = 3, col = col)
  } else {
    lcl <- predictionsPlusCI$fit[ord.index] - 1.96*predictionsPlusCI$se.fit[ord.index]
    ucl <- predictionsPlusCI$fit[ord.index] + 1.96*predictionsPlusCI$se.fit[ord.index]
    
    lines(x = xx, y = lcl, lty = 2, lwd = 2, col = col)
    lines(x = xx, y = ucl, lty = 2, lwd = 2, col = col)
    lines(xx, predictionsPlusCI$fit[ord.index], lwd = 3, col = col)
  }
  
  if(!missing(label)) fig_label(label)
}

# prop plot MAP 24h and hospital mortality, ICU and AKI

In [11]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_map_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_map_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_map_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)


In [13]:
pdf(file = "figure/gradient/prop_7d_MAP24_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 24h", label = "A1",bp_type = "MAP",bp_low = 70, bp_high = 80)
gam_plotProp(gammfitProp_eICU_2, main = "eICU 24h", label = "B1",bp_type = "MAP",bp_low = 70, bp_high = 80)
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 24h", label = "C1", bp_type = "MAP", bp_low = 70,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 24h", label = "A2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 24h", label = "B2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 24h", label = "C2",col ="blue",bp_type = "MAP", bp_low = 70, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 24h", label = "A3", bp_type = "MAP", bp_low =70, bp_high = 80)
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 24h", label = "B3", bp_type = "MAP", bp_low = 70, bp_high = 80)
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 24h", label = "C3", bp_type = "MAP", bp_low = 70)


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 24h", label = "A4",bp_type = "MAP", col ="blue", bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 24h", label = "B4",bp_type = "MAP",col ="blue", bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 24h", label = "C4",bp_type = "MAP",col ="blue", bp_low = 70)


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

# prop plot MAP 48h and hospital mortality, ICU and AKI

In [14]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_map_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_map_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_map_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)


In [15]:
pdf(file = "figure/gradient/prop_7d_MAP48_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 48h", label = "A1",bp_type = "MAP",bp_low = 70, bp_high = 80)
gam_plotProp(gammfitProp_eICU_2, main = "eICU 48h", label = "B1",bp_type = "MAP",bp_low = 70, bp_high = 80)
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 48h", label = "C1", bp_type = "MAP", bp_low = 70,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 48h", label = "A2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 48h", label = "B2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 48h", label = "C2",col ="blue",bp_type = "MAP", bp_low = 70, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 48h", label = "A3", bp_type = "MAP", bp_low =70, bp_high = 80)
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 48h", label = "B3", bp_type = "MAP", bp_low = 70, bp_high = 80)
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 48h", label = "C3", bp_type = "MAP", bp_low = 70,yRange = c(0.6, 1.0))


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 48h", label = "A4",bp_type = "MAP", col ="blue", bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 48h", label = "B4",bp_type = "MAP",col ="blue", bp_low = 70, bp_high = 80)
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 48h", label = "C4",bp_type = "MAP",col ="blue", bp_low = 70,yRange = c(0.6, 1.0))


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

# prop plot MAP 72h and hospital mortality, ICU and AKI

In [16]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_map_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_map_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_map_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)


In [17]:
pdf(file = "figure/gradient/prop_7d_MAP72_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 72h", label = "A1",bp_type = "MAP",bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp(gammfitProp_eICU_2, main = "eICU 72h", label = "B1",bp_type = "MAP",bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 72h", label = "C1", bp_type = "MAP", bp_low = 70,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 72h", label = "A2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80, yRange = c(0,0.30))
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 72h", label = "B2",bp_type = "MAP",col ="blue",bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 72h", label = "C2",col ="blue",bp_type = "MAP", bp_low = 70, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 72h", label = "A3", bp_type = "MAP", bp_low =70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 72h", label = "B3", bp_type = "MAP", bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 72h", label = "C3", bp_type = "MAP", bp_low = 70,yRange = c(0.6,1.0))


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 72h", label = "A4",bp_type = "MAP", col ="blue", bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 72h", label = "B4",bp_type = "MAP",col ="blue", bp_low = 70, bp_high = 80,yRange = c(0,0.30))
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 72h", label = "C4",bp_type = "MAP",col ="blue", bp_low = 70,yRange = c(0.6,1.0))


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot SBP 24h and hospital mortality, ICU and AKI

In [18]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_asp_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_asp_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_asp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)


In [19]:
pdf(file = "figure/gradient/prop_7d_SBP24_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 24h", label = "A1",bp_type = "SBP",bp_low = 110, bp_high = 120)
gam_plotProp(gammfitProp_eICU_2, main = "eICU 24h", label = "B1",bp_type = "SBP",bp_low = 110, bp_high = 120)
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 24h", label = "C1", bp_type = "SBP", bp_low = 110,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 24h", label = "A2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 24h", label = "B2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120)
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 24h", label = "C2",col ="blue",bp_type = "SBP", bp_low = 110, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 24h", label = "A3", bp_type = "SBP", bp_low =110, bp_high = 120)
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 24h", label = "B3", bp_type = "SBP", bp_low = 110, bp_high = 120)
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 24h", label = "C3", bp_type = "SBP", bp_low = 110)


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 24h", label = "A4",bp_type = "SBP", col ="blue", bp_low = 110, bp_high = 120)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 24h", label = "B4",bp_type = "SBP",col ="blue", bp_low = 110, bp_high = 120)
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 24h", label = "C4",bp_type = "SBP",col ="blue", bp_low = 110)


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot SBP 48h and hospital mortality, ICU and AKI

In [20]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_asp_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_asp_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_asp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)


In [21]:
pdf(file = "figure/gradient/prop_7d_SBP48_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 48h", label = "A1",bp_type = "SBP",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gammfitProp_eICU_2, main = "eICU 48h", label = "B1",bp_type = "SBP",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 48h", label = "C1", bp_type = "SBP", bp_low = 110,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 48h", label = "A2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 48h", label = "B2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 48h", label = "C2",col ="blue",bp_type = "SBP", bp_low = 110, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 48h", label = "A3", bp_type = "SBP", bp_low =110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 48h", label = "B3", bp_type = "SBP", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 48h", label = "C3", bp_type = "SBP", bp_low = 110)


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 48h", label = "A4",bp_type = "SBP", col ="blue", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 48h", label = "B4",bp_type = "SBP",col ="blue", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.25))
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 48h", label = "C4",bp_type = "SBP",col ="blue", bp_low = 110)


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot SBP 72h and hospital mortality, ICU and AKI

In [22]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_asp_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_asp_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_asp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)


In [23]:
pdf(file = "figure/gradient/prop_7d_SBP72_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 72h", label = "A1",bp_type = "SBP",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gammfitProp_eICU_2, main = "eICU 72h", label = "B1",bp_type = "SBP",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 72h", label = "C1", bp_type = "SBP", bp_low = 70,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 72h", label = "A2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 72h", label = "B2",bp_type = "SBP",col ="blue",bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 72h", label = "C2",col ="blue",bp_type = "SBP", bp_low = 110, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 72h", label = "A3", bp_type = "SBP", bp_low =110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 72h", label = "B3", bp_type = "SBP", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 72h", label = "C3", bp_type = "SBP", bp_low = 110)


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 72h", label = "A4",bp_type = "SBP", col ="blue", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 72h", label = "B4",bp_type = "SBP",col ="blue", bp_low = 110, bp_high = 120,yRange = c(0.0, 0.3))
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 72h", label = "C4",bp_type = "SBP",col ="blue", bp_low = 110)


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot DBP 24h and hospital mortality, ICU and AKI

In [24]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_adp_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_adp_24_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_adp_24)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI, family = binomial)


In [25]:
pdf(file = "figure/gradient/prop_7d_DBP24_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 24h", label = "A1",bp_type = "DBP",bp_low = 50, bp_high = 60)
gam_plotProp(gammfitProp_eICU_2, main = "eICU 24h", label = "B1",bp_type = "DBP",bp_low = 50, bp_high = 60)
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 24h", label = "C1", bp_type = "DBP", bp_low = 50,yRange = c(0.75, 0.9))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 24h", label = "A2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60)
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 24h", label = "B2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60)
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 24h", label = "C2",col ="blue",bp_type = "DBP", bp_low = 50, yRange = c(0.75, 0.9))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 24h", label = "A3", bp_type = "DBP", bp_low =50, bp_high = 60)
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 24h", label = "B3", bp_type = "DBP", bp_low = 50, bp_high = 60)
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 24h", label = "C3", bp_type = "DBP", bp_low = 50)


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 24h", label = "A4",bp_type = "DBP", col ="blue", bp_low = 50, bp_high = 60)
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 24h", label = "B4",bp_type = "DBP",col ="blue", bp_low = 50, bp_high = 60)
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 24h", label = "C4",bp_type = "DBP",col ="blue", bp_low = 50)


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot DBP 48h and hospital mortality, ICU and AKI

In [26]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_48, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_48, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_adp_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_48, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_adp_48_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_adp_48)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_48, family = binomial)


In [27]:
pdf(file = "figure/gradient/prop_7d_DBP48_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 48h", label = "A1",bp_type = "DBP",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gammfitProp_eICU_2, main = "eICU 48h", label = "B1",bp_type = "DBP",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 48h", label = "C1", bp_type = "DBP", bp_low = 50,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 48h", label = "A2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 48h", label = "B2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 48h", label = "C2",col ="blue",bp_type = "DBP", bp_low = 50, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 48h", label = "A3", bp_type = "DBP", bp_low =50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 48h", label = "B3", bp_type = "DBP", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 48h", label = "C3", bp_type = "DBP", bp_low = 50,yRange = c(0.6, 1.0))


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 48h", label = "A4",bp_type = "DBP", col ="blue", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 48h", label = "B4",bp_type = "DBP",col ="blue", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 48h", label = "C4",bp_type = "DBP",col ="blue", bp_low = 50,yRange = c(0.6, 1.0))


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

## proportion plot DBP 72h and hospital mortality, ICU and AKI

In [28]:
# hospital
gammfitProp_eICU <- gam(hospital_expire_flag ~ s(prop_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU <- gam(hospital_expire_flag ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU <- gam(hospital_expire_flag ~ s(propAbove_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC <- gam(hospital_expire_flag ~ s(prop_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC <- gam(hospital_expire_flag ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC <- gam(hospital_expire_flag ~ s(propAbove_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
# icu
gammfitProp_eICU_2 <- gam(icu_mortality ~ s(prop_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_below_eICU_2 <- gam(icu_mortality ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_above_eICU_2 <- gam(icu_mortality ~ s(propAbove_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_72, family = binomial)
gamfitProp_MIMIC_2 <- gam(icu_mortality ~ s(prop_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_below_MIMIC_2 <- gam(icu_mortality ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)
gamfitProp_above_MIMIC_2 <- gam(icu_mortality ~ s(propAbove_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_72, family = binomial)

#AKI
gamfitProp_eICU_AKI <- gam(AKI_7d ~ s(prop_adp_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_below_eICU_AKI <- gam(AKI_7d ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_eicu_prop_AKI_72, family = binomial)
gamfitProp_MIMIC_AKI <- gam(AKI_7d ~ s(prop_adp_72_AKI)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)
gamfitProp_below_MIMIC_AKI <- gam(AKI_7d ~ s(propBelow_adp_72)+gender+s(age)+s(bmi)+s(sofatotal), data = df_mimic_prop_AKI_72, family = binomial)


In [29]:
pdf(file = "figure/gradient/prop_7d_DBP72_Hosp_ICU_AKI.pdf", width = 16,
    height = 9, pointsize = point_size, family = font)
par(mfcol = c(3,4), cex = 1)

gam_plotProp(gammfitProp_eICU, main = "eICU 72h", label = "A1",bp_type = "DBP",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gammfitProp_eICU_2, main = "eICU 72h", label = "B1",bp_type = "DBP",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp_AKI(gamfitProp_eICU_AKI, main = "eICU 72h", label = "C1", bp_type = "DBP", bp_low = 50,yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_below_eICU, main = "eICU 72h", label = "A2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_eICU, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_eICU_2, main = "eICU 72h", label = "B2",bp_type = "DBP",col ="blue",bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_eICU_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_eICU_AKI, main = "eICU 72h", label = "C2",col ="blue",bp_type = "DBP", bp_low = 50, yRange = c(0.75, 0.95))


gam_plotProp(gamfitProp_MIMIC, main = "MIMIC 72h", label = "A3", bp_type = "DBP", bp_low =50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_MIMIC_2, main = "MIMIC 72h", label = "B3", bp_type = "DBP", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp_AKI(gamfitProp_MIMIC_AKI, main = "MIMIC 72h", label = "C3", bp_type = "DBP", bp_low = 50,yRange = c(0.6, 1.0))


gam_plotProp(gamfitProp_below_MIMIC, main = "MIMIC 72h", label = "A4",bp_type = "DBP", col ="blue", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_MIMIC, main = "", col = "red", add = TRUE)
gam_plotProp(gamfitProp_below_MIMIC_2, main = "MIMIC 72h", label = "B4",bp_type = "DBP",col ="blue", bp_low = 50, bp_high = 60,yRange = c(0.0, 0.2))
gam_plotProp(gamfitProp_above_MIMIC_2, main = "", col = "red", add = TRUE)
gam_plotProp_AKI(gamfitProp_below_MIMIC_AKI, main = "MIMIC 72h", label = "C4",bp_type = "DBP",col ="blue", bp_low = 50,yRange = c(0.6, 1.0))


dev.off()


[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 14374"
[1] "Number of cases: 13105"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 12291"
[1] "Number of cases: 11445"


png 
  2

# Find optimal range

In [23]:
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_24h)
head(eicu_grad)

bp_value,gradient,pre_val_left,pre_val_right
38.5,-0.8256371,0.3106166,0.2734630
40.0,-0.7827562,0.2734630,0.2617216
41.0,-0.7642606,0.2617216,0.2540790
42.5,-0.7459086,0.2540790,0.2428904
44.0,-0.7245297,0.2428904,0.2320224
45.0,-0.7076894,0.2320224,0.2249455


In [57]:
print("eICU") #73-79
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_24h) #73-75, 69-78
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_48h) #76-79, 73-81
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_MAP_72h)#77-79, 74-83
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
print("MIMIC") #72-75
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_24h)#72-74, 69-75
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_48h)#73-75, 70-76
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_MAP_72h) #74-75, 71-76
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]


[1] "eICU"


[1] 71 72 73 74 75

[1] 75 76 77 78

[1] 75 76 77 78 79

[1] "MIMIC"


[1] 71 72 73

[1] 71 72 73 74

[1] 73 74

## systolic vs hospital mortality

In [58]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_24h) #112-120
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_48h) #116-124
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ASP_72h)#118-125
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
print("MIMIC") #112-120
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_24h)#112-114
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_48h)#115-118
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ASP_72h) #115-120
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]


[1] "eICU"


[1] 111 112 113 114 115 116 117 118 119 120

[1] 114 115 116 117 118 119 120 121 122 123 124

[1] 116 117 118 119 120 121 122 123 124 125 126

[1] "MIMIC"


[1] 110 111 112 113 114

[1] 113 114 115 116 117 118

[1] 113 114 115 116 117 118 119 120

## diastolic vs hospital mortality

In [59]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_24h) #50-54
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_48h) #53-59
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_eICU_ADP_72h)#54-61
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
print("MIMIC") #41-59
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_24h)#41-52
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_48h)#43-54
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_MIMIC_ADP_72h) #47-59
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]


[1] "eICU"


[1] 48 49 50 51 52 53 54

[1] 52 53 54 55 56 57 58

[1] 53 54 55 56 57 58 59 60 61

[1] "MIMIC"


[1] 41 42 43 44 45 46 47 48 49 50 51 52

[1] 41 42 43 44 45 46 47 48 49 50 51 52 53 54

[1] 46 47 48 51 52 53 54 55 56 57 58

## mean vs ICU mortality

In [60]:
print("eICU") #73-80
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_24h) #73-75
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_48h) #77-79
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_MAP_72h)#78-80
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value< 90) & (eicu_grad$bp_value> 60), "bp_value"]
print("MIMIC") #73-77
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_24h)#73-74
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_48h)#74-77
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_MAP_72h) #75-77
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value< 90) & (mimic_grad$bp_value> 60), "bp_value"]


[1] "eICU"


[1] 71 72 73 74 75 76

[1] 75 76 77 78 79 80

[1] 76 77 78 79 80 81

[1] "MIMIC"


[1] 71 72 73 74

[1] 72 73 74 75 76

[1] 73 74 75 76 89

## systolic vs ICU mortality

In [61]:
print("eICU") #112-125
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_24h) #113-120
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_48h) #116-126
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ASP_72h)#118-128
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value> 90) & (eicu_grad$bp_value< 140), "bp_value"]
print("MIMIC") #112-120
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_24h)#113-115
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_48h)#116-119
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ASP_72h) #116-122
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value> 90) & (mimic_grad$bp_value<140), "bp_value"]


[1] "eICU"


[1] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124

[1] 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131
[20] 132 133 134

[1] 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132
[20] 133 134 135 136 137 138 139

[1] "MIMIC"


[1] 110 111 112 113 114 115 116 139

[1] 113 114 115 116 117 118 119 120

[1] 114 115 116 117 118 119 120 121 122 123

## diastolic vs ICU mortality

In [62]:
print("eICU") #49-65
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_24h) #49-54
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_48h) #54-61
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1)
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
eicu_grad <- find_gradient(b = gamfitMed_ICU_eICU_ADP_72h)#56-65
eicu_grad[(eicu_grad$gradient > -0.1) & (eicu_grad$gradient< 0.1) 
                 & (eicu_grad$bp_value> 40) & (eicu_grad$bp_value< 70), "bp_value"]
print("MIMIC") #41-60
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_24h)#41-54
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_48h)#46-56
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1) 
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]
mimic_grad <- find_gradient(b = gamfitMed_ICU_MIMIC_ADP_72h) #47-60
mimic_grad[(mimic_grad$gradient > -0.1) & (mimic_grad$gradient< 0.1)
                 & (mimic_grad$bp_value> 40) & (mimic_grad$bp_value<70), "bp_value"]


[1] "eICU"


[1] 46 47 48 49 50 51 52 53 54 55 56

[1] 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64

[1] 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68

[1] "MIMIC"


[1] 41 42 43 44 45 46 47 48 49 50 51 52 53 54

[1] 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57

[1] 46 47 51 52 53 59 60

## mean, systolic, diastolic vs AKI (7day)

In [44]:
print("eICU") # NA
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_24h) 
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_48h) 
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_MAP_72h)
eicu_grad[(eicu_grad$bp_value< 80) & (eicu_grad$bp_value> 70),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_24h)
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_48h)         #75
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_MAP_72h)         #76
mimic_grad[(mimic_grad$bp_value< 80) & (mimic_grad$bp_value> 70),]


[1] "eICU"


,bp_value,gradient,pre_val_left,pre_val_right
30,71,-0.1346184,0.8602618,0.8589156
31,72,-0.1357088,0.8589156,0.8575585
32,73,-0.1367414,0.8575585,0.8561911
33,74,-0.1376239,0.8561911,0.8548148
34,75,-0.1382507,0.8548148,0.8534323
35,76,-0.1385073,0.8534323,0.8520473
36,77,-0.1382763,0.8520473,0.8506645
37,78,-0.1374497,0.8506645,0.8492900
38,79,-0.1359363,0.8492900,0.8479306


,bp_value,gradient,pre_val_left,pre_val_right
26,71,-0.1717180,0.8634645,0.8617473
27,72,-0.1731429,0.8617473,0.8600159
28,73,-0.1745055,0.8600159,0.8582708
29,74,-0.1757323,0.8582708,0.8565135
30,75,-0.1767448,0.8565135,0.8547461
31,76,-0.1774454,0.8547461,0.8529716
32,77,-0.1777110,0.8529716,0.8511945
33,78,-0.1774219,0.8511945,0.8494203
34,79,-0.1764719,0.8494203,0.8476556


,bp_value,gradient,pre_val_left,pre_val_right
25,71,-0.1474253,0.8631683,0.8616941
26,72,-0.1506125,0.8616941,0.8601879
27,73,-0.1538716,0.8601879,0.8586492
28,74,-0.1570957,0.8586492,0.8570783
29,75,-0.1601750,0.8570783,0.8554765
30,76,-0.1629778,0.8554765,0.8538467
31,77,-0.1653721,0.8538467,0.8521930
32,78,-0.1672202,0.8521930,0.8505208
33,79,-0.1683547,0.8505208,0.8488373


[1] "MIMIC"


,bp_value,gradient,pre_val_left,pre_val_right
31,71,-0.3029696,0.7741148,0.7710851
32,72,-0.3050684,0.7710851,0.7680344
33,73,-0.3056293,0.7680344,0.7649781
34,74,-0.3044665,0.7649781,0.7619335
35,75,-0.3014401,0.7619335,0.7589191
36,76,-0.2964638,0.7589191,0.7559544
37,77,-0.2895104,0.7559544,0.7530593
38,78,-0.2806051,0.7530593,0.7502533
39,79,-0.2698120,0.7502533,0.7475552


,bp_value,gradient,pre_val_left,pre_val_right
22,71,0.12025472,0.7881753,0.7893779
23,72,0.02496284,0.7893779,0.7896275
24,73,-0.11458472,0.7896275,0.7884817
25,74,-0.28145942,0.7884817,0.7856671
26,75,-0.45859730,0.7856671,0.7810811
27,76,-0.63048495,0.7810811,0.7747763
28,77,-0.78345756,0.7747763,0.7669417
29,78,-0.90244454,0.7669417,0.7579172
30,79,-0.97465814,0.7579172,0.7481707


,bp_value,gradient,pre_val_left,pre_val_right
22,71,0.08988006,0.7837175,0.7846163
23,72,0.06278884,0.7846163,0.7852442
24,73,-0.01287540,0.7852442,0.7851155
25,74,-0.12757638,0.7851155,0.7838397
26,75,-0.26914529,0.7838397,0.7811483
27,76,-0.42472946,0.7811483,0.7769010
28,77,-0.58122016,0.7769010,0.7710888
29,78,-0.72537937,0.7710888,0.7638350
30,79,-0.84499241,0.7638350,0.7553850


In [47]:
print("eICU") # NA
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_24h) 
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_48h) 
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]
eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ASP_72h)
eicu_grad[(eicu_grad$bp_value< 120) & (eicu_grad$bp_value> 110),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_24h)
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 110),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_48h)         #111
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 100),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ASP_72h)         # NA
mimic_grad[(mimic_grad$bp_value< 120) & (mimic_grad$bp_value> 100),]


[1] "eICU"


,bp_value,gradient,pre_val_left,pre_val_right
50,111,-0.05985599,0.8575447,0.8569461
51,112,-0.06006674,0.8569461,0.8563454
52,113,-0.06027764,0.8563454,0.8557426
53,114,-0.06048865,0.8557426,0.8551378
54,115,-0.06069972,0.8551378,0.8545308
55,116,-0.06091083,0.8545308,0.8539217
56,117,-0.06112193,0.8539217,0.8533104
57,118,-0.06133300,0.8533104,0.8526971
58,119,-0.06154398,0.8526971,0.8520817


,bp_value,gradient,pre_val_left,pre_val_right
46,111,-0.07311897,0.8585067,0.8577755
47,112,-0.07343442,0.8577755,0.8570411
48,113,-0.07375053,0.8570411,0.8563036
49,114,-0.07406728,0.8563036,0.8555629
50,115,-0.07438468,0.8555629,0.8548191
51,116,-0.07470269,0.8548191,0.8540721
52,117,-0.07502133,0.8540721,0.8533219
53,118,-0.07534057,0.8533219,0.8525685
54,119,-0.07566042,0.8525685,0.8518118


,bp_value,gradient,pre_val_left,pre_val_right
46,111,-0.05956893,0.8567534,0.8561577
47,112,-0.05977531,0.8561577,0.8555600
48,113,-0.05998206,0.8555600,0.8549602
49,114,-0.06018918,0.8549602,0.8543583
50,115,-0.06039664,0.8543583,0.8537543
51,116,-0.06060446,0.8537543,0.8531483
52,117,-0.06081261,0.8531483,0.8525401
53,118,-0.06102111,0.8525401,0.8519299
54,119,-0.06122993,0.8519299,0.8513176


[1] "MIMIC"


,bp_value,gradient,pre_val_left,pre_val_right
40,111,-0.3243964,0.7716155,0.7683716
41,112,-0.3240273,0.7683716,0.7651313
42,113,-0.3219798,0.7651313,0.7619115
43,114,-0.3182074,0.7619115,0.7587294
44,115,-0.3126926,0.7587294,0.7556025
45,116,-0.3054472,0.7556025,0.7525480
46,117,-0.2965134,0.7525480,0.7495829
47,118,-0.2859631,0.7495829,0.7467233
48,119,-0.2738971,0.7467233,0.7439843


,bp_value,gradient,pre_val_left,pre_val_right
25,101,-0.14061559,0.7937501,0.7923440
26,102,-0.10806373,0.7923440,0.7912634
27,103,-0.08253439,0.7912634,0.7904380
28,104,-0.06577934,0.7904380,0.7897802
29,105,-0.05909169,0.7897802,0.7891893
30,106,-0.06318202,0.7891893,0.7885575
31,107,-0.07819391,0.7885575,0.7877755
32,108,-0.10387410,0.7877755,0.7867368
33,109,-0.13927960,0.7867368,0.7853440
34,110,-0.18315877,0.7853440,0.7835124


,bp_value,gradient,pre_val_left,pre_val_right
24,101,-0.1462012,0.7945428,0.7930808
25,102,-0.1360190,0.7930808,0.7917206
26,103,-0.1296856,0.7917206,0.7904237
27,104,-0.1279844,0.7904237,0.7891439
28,105,-0.1315485,0.7891439,0.7878284
29,106,-0.1407606,0.7878284,0.7864208
30,107,-0.1557066,0.7864208,0.7848637
31,108,-0.1761704,0.7848637,0.7831020
32,109,-0.2016260,0.7831020,0.7810858
33,110,-0.2312940,0.7810858,0.7787728


In [49]:
# print("eICU") # NA
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_24h) 
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_48h) 
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]
# eicu_grad <- find_gradient(b = gamfitMed_AKI_eICU_ADP_72h)
# eicu_grad[(eicu_grad$bp_value< 60) & (eicu_grad$bp_value> 50),]

print("MIMIC") # 75mmHg, 76mmHg
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_24h)
mimic_grad[(mimic_grad$bp_value< 60) & (mimic_grad$bp_value> 50),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_48h)         #58
mimic_grad[(mimic_grad$bp_value< 70) & (mimic_grad$bp_value> 50),]
mimic_grad <- find_gradient(b = gamfitMed_AKI_MIMIC_ADP_72h)         #58
mimic_grad[(mimic_grad$bp_value< 70) & (mimic_grad$bp_value> 50),]


[1] "MIMIC"


,bp_value,gradient,pre_val_left,pre_val_right
28,51,-0.2137301,0.7585594,0.7564221
29,52,-0.2066908,0.7564221,0.7543552
30,53,-0.1992903,0.7543552,0.7523623
31,54,-0.1914031,0.7523623,0.7504483
32,55,-0.1828654,0.7504483,0.7486196
33,56,-0.1734939,0.7486196,0.7468847
34,57,-0.1630812,0.7468847,0.7452539
35,58,-0.1514587,0.7452539,0.7437393
36,59,-0.1385037,0.7437393,0.7423543


,bp_value,gradient,pre_val_left,pre_val_right
22,51,0.001802263,0.7609551,0.7609731
23,52,-0.003072151,0.7609731,0.7609424
24,53,-0.015833902,0.7609424,0.7607840
25,54,-0.039986802,0.7607840,0.7603842
26,55,-0.080893074,0.7603842,0.7595752
27,56,-0.141652336,0.7595752,0.7581587
28,57,-0.222758197,0.7581587,0.7559311
29,58,-0.320240302,0.7559311,0.7527287
30,59,-0.423506500,0.7527287,0.7484936
31,60,-0.521524437,0.7484936,0.7432784


,bp_value,gradient,pre_val_left,pre_val_right
21,51,-0.26725026,0.7644893,0.7618168
22,52,-0.18262335,0.7618168,0.7599906
23,53,-0.12492383,0.7599906,0.7587413
24,54,-0.09740732,0.7587413,0.7577673
25,55,-0.09996291,0.7577673,0.7567676
26,56,-0.12988791,0.7567676,0.7554688
27,57,-0.18127050,0.7554688,0.7536561
28,58,-0.24568426,0.7536561,0.7511992
29,59,-0.31476682,0.7511992,0.7480516
30,60,-0.38142549,0.7480516,0.7442373


# Hospital/ICU Mortality odds ratio for MBP 5-15% vs 35-45% in-range

In [530]:
library(epitools)

In [63]:
head(df_eicu_prop, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,use_vasopressor,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,0,0.9583333,0.0000000,0.04166667,0.7500000,0.04166667,0.2083333,1.0000000,0.000000000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,1,0.3457944,0.1495327,0.50467290,0.3271028,0.45794393,0.2149533,0.9065421,0.009345794,0.08411215


### 24h eicu

In [ ]:

a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
c<-length(df_10[df_10$hospital_expire_flag==0,1])
b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
d<-length(df_40[df_40$hospital_expire_flag==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))


In [392]:
criteria <-df_eicu_prop$ihd==1

In [393]:
df_10<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.1 & df_eicu_prop$prop_map_24<=0.3 & criteria,]
df_40<-df_eicu_prop[df_eicu_prop$prop_map_24>=0.3 & df_eicu_prop$prop_map_24<=0.5 & criteria,]

In [395]:
df_10<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.1 & df_eicu_prop$prop_asp_24<=0.3 & criteria,]
df_40<-df_eicu_prop[df_eicu_prop$prop_asp_24>=0.3 & df_eicu_prop$prop_asp_24<=0.5 & criteria,]

In [397]:
df_10<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.1 & df_eicu_prop$prop_adp_24<=0.3 & criteria,]
df_40<-df_eicu_prop[df_eicu_prop$prop_adp_24>=0.3 & df_eicu_prop$prop_adp_24<=0.5 & criteria,]

In [398]:
a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
c<-length(df_10[df_10$icu_mortality==0,1])
b<-length(df_40[df_40$icu_mortality==1,1]) # control
d<-length(df_40[df_40$icu_mortality==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

[1]   3  12  96 163


[1] 0.03030303

[1] 0.06857143

,Disease1,Disease2,Total
Exposed1,3,96,99
Exposed2,12,163,175
Total,15,259,274
,estimate,lower,upper
Exposed1,1.0000000,NA,NA
Exposed2,0.4415322,0.09421387,1.454054
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.1906069,0.2694999,0.1809963


In [612]:
c1 <- df_mimic_prop$vasso_duration_24h>0
c2 <- df_mimic_prop$unittype=='MICU'
c3 <- df_mimic_prop$unittype=='Cardiac ICU'
c4 <- df_mimic_prop$unittype=='CSICU'
c5 <- df_mimic_prop$unittype=='Med-Surg ICU'
c6 <- df_mimic_prop$unittype=='CTICU'
Criterias <- data.frame("C4"=c1)
# Criterias <- data.frame("c1"=c1, "c2"=c2,"C3" = c3, "C4"=c4, "C5" = c5, "C6"=c6)
Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    
    # MAP48
    df_10<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.05 & df_mimic_prop$prop_map_24<=0.15 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.35 & df_mimic_prop$prop_map_24<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]
    
   
    # SBP48
    df_10<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.05 & df_mimic_prop$prop_asp_24<=0.15 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.35 & df_mimic_prop$prop_asp_24<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+4]<-df_OR$measure[2]
    Hosp_LCI[j+4]<-df_OR$measure[4]
    Hosp_UCI[j+4]<-df_OR$measure[6]
    Hosp_pvalue[j+4]<-df_OR$p.value[4]
    
   
    # DBP48
    df_10<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.1 & df_mimic_prop$prop_adp_24<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.3 & df_mimic_prop$prop_adp_24<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+7]<-df_OR$measure[2]
    Hosp_LCI[j+7]<-df_OR$measure[4]
    Hosp_UCI[j+7]<-df_OR$measure[6]
    Hosp_pvalue[j+7]<-df_OR$p.value[4]
   
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+9
    
}

In [613]:
df_OR_CI

OR,LCI,UCI,pvalue
2.394042,1.827023,3.129435,5.115070e-10
1.886656,1.386548,2.612293,4.276908e-05
1.394144,1.149497,1.689360,7.831663e-04


In [ ]:
c1 <- df_mimic_prop$copd==1
# c2 <- df_mimic_prop$stroke==0
# c3 <- df_mimic_prop$ihd==1
# c4 <- df_mimic_prop$diabetes==1
# c5 <- df_mimic_prop$cancer==0
# c6 <- df_mimic_prop$bmi<30
# Criterias <- data.frame("C4"=c4)
# Criterias <- data.frame("c1"=c1, "c2"=c2)
# 
# , "C4"=c4,  "C5" = c5, "C6"=c6 )
# Criterias <- data.frame("c1"=c1,"c2"=c2,"C3" = c3, "c4"=c4, "C5" = c5 , "c6"=c6)
Criterias <- data.frame("C3" = c1)
Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.1 & df_mimic_prop$prop_map_24<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.3 & df_mimic_prop$prop_map_24<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]
    # MAP48
    df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_map_48>=0.1 & df_mimic_prop_48$prop_map_48<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_map_48>=0.3 & df_mimic_prop_48$prop_map_48<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]
    # MAP72
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_map_72>=0.1 & df_mimic_prop_72$prop_map_72<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_map_72>=0.3 & df_mimic_prop_72$prop_map_72<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]
    # SBP 24
    df_10<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.1 & df_mimic_prop$prop_asp_24<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.3 & df_mimic_prop$prop_asp_24<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+3]<-df_OR$measure[2]
    Hosp_LCI[j+3]<-df_OR$measure[4]
    Hosp_UCI[j+3]<-df_OR$measure[6]
    Hosp_pvalue[j+3]<-df_OR$p.value[4]
    
    # SBP48
    df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_asp_48>=0.1 & df_mimic_prop_48$prop_asp_48<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_asp_48>=0.3 & df_mimic_prop_48$prop_asp_48<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+4]<-df_OR$measure[2]
    Hosp_LCI[j+4]<-df_OR$measure[4]
    Hosp_UCI[j+4]<-df_OR$measure[6]
    Hosp_pvalue[j+4]<-df_OR$p.value[4]
    
    # SBP72
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.1 & df_mimic_prop_72$prop_asp_72<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.3 & df_mimic_prop_72$prop_asp_72<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+5]<-df_OR$measure[2]
    Hosp_LCI[j+5]<-df_OR$measure[4]
    Hosp_UCI[j+5]<-df_OR$measure[6]
    Hosp_pvalue[j+5]<-df_OR$p.value[4]
    # DBP 24
    df_10<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.1 & df_mimic_prop$prop_adp_24<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.3 & df_mimic_prop$prop_adp_24<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+6]<-df_OR$measure[2]
    Hosp_LCI[j+6]<-df_OR$measure[4]
    Hosp_UCI[j+6]<-df_OR$measure[6]
    Hosp_pvalue[j+6]<-df_OR$p.value[4]
    # DBP48
    df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.1 & df_mimic_prop_48$prop_adp_48<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.3 & df_mimic_prop_48$prop_adp_48<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+7]<-df_OR$measure[2]
    Hosp_LCI[j+7]<-df_OR$measure[4]
    Hosp_UCI[j+7]<-df_OR$measure[6]
    Hosp_pvalue[j+7]<-df_OR$p.value[4]
    # DBP49
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.1 & df_mimic_prop_72$prop_adp_72<=0.3 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.3 & df_mimic_prop_72$prop_adp_72<=0.5 & Criterias[,i],]
    a<-length(df_10[df_10$icu_mortality==1,1]) # exposed
    c<-length(df_10[df_10$icu_mortality==0,1])
    b<-length(df_40[df_40$icu_mortality==1,1]) # control
    d<-length(df_40[df_40$icu_mortality==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+8]<-df_OR$measure[2]
    Hosp_LCI[j+8]<-df_OR$measure[4]
    Hosp_UCI[j+8]<-df_OR$measure[6]
    Hosp_pvalue[j+8]<-df_OR$p.value[4]
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+9
    
}

In [1364]:
c1 <- df_mimic_prop_72$sepsis==1
c2 <- df_mimic_prop_72$sepsis==0
# c3 <- df_mimic_prop_72$ihd==1
# c4 <- df_mimic_prop_72$ihd==0

# c5 <- df_mimic_prop_72$cancer==0
# c6 <- df_mimic_prop_72$unittype=="SICU"
# c7 <- df_mimic_prop_72$unittype=="MICU"
# c8<-  df_mimic_prop_72$unittype=="Cardiac ICU"
# c9<-  df_mimic_prop_72$unittype=="CSICU"
# c10<-  df_mimic_prop_72$unittype=="Med-Surg ICU"
# c11<- df_mimic_prop_72$unittype=="CTICU"
# c12<- df_mimic_prop_72$unittype=="CCU-CTICU"
# c13<-df_mimic_prop_72$unittype=="Neuro ICU"
# Criterias <- data.frame("c2"=c4)
# Criterias <- data.frame("c1"=c1,"c2"=c2,"C3" = c3, "c4"=c4, "C5" = c5 , "c6"=c6,"c7"=c7, "c8"=c8,"c9"=c9,
#                         "c10"=c10,"c11"=c11, "c12"=c12,"c13"=c13)
Criterias <- data.frame( "c1"=c1,"c2"=c2)
# Criterias <- data.frame( "c2"=c03)
Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_map_72>=0.05 & df_mimic_prop_72$prop_map_72<=0.15 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_map_72>=0.35 & df_mimic_prop_72$prop_map_72<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.05 & df_mimic_prop_72$prop_asp_72<=0.15 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.35 & df_mimic_prop_72$prop_asp_72<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.05 & df_mimic_prop_72$prop_adp_72<=0.15 & Criterias[,i],]
    df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.35 & df_mimic_prop_72$prop_adp_72<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]

    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+3
    
}

In [1365]:
df_OR_CI

OR,LCI,UCI,pvalue
1.0201620,0.7934406,1.3075435,8.986469e-01
1.3203193,0.9399969,1.8878499,1.310815e-01
0.7013307,0.5305482,0.9217588,1.174318e-02
2.5963629,2.0014503,3.3737316,9.362903e-13
2.8453902,1.9150923,4.4110251,3.119524e-08
1.7697224,1.3363373,2.3417451,7.991051e-05


### 24h mimic

In [343]:
criteria <- df_mimic_prop$ethnicity =='ASIAN'

In [344]:
df_10<-df_mimic_prop[df_mimic_prop$prop_map_24>=0.1 & df_mimic_prop$prop_asp_24<=0.3 & criteria ,]
df_40<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.3 & df_mimic_prop$prop_asp_24<=0.5 & criteria ,]

In [346]:
df_10<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.1 & df_mimic_prop$prop_asp_24<=0.3 & criteria ,]
df_40<-df_mimic_prop[df_mimic_prop$prop_asp_24>=0.3 & df_mimic_prop$prop_asp_24<=0.5 & criteria ,]

In [348]:
df_10<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.1 & df_mimic_prop$prop_adp_24<=0.3 & criteria ,]
df_40<-df_mimic_prop[df_mimic_prop$prop_adp_24>=0.3 & df_mimic_prop$prop_adp_24<=0.5 & criteria ,]

### 48h eicu

In [399]:
df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_map_48>=0.1 & df_eicu_prop_48$prop_map_48<=0.3 & criteria ,]
df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_map_48>=0.3 & df_eicu_prop_48$prop_map_48<=0.5 & criteria ,]

In [401]:
df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_asp_48>=0.1 & df_eicu_prop_48$prop_asp_48<=0.3 & criteria ,]
df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_asp_48>=0.3 & df_eicu_prop_48$prop_asp_48<=0.5 & criteria ,]

In [403]:
df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_adp_48>=0.1 & df_eicu_prop_48$prop_adp_48<=0.3 & criteria ,]
df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_adp_48>=0.3 & df_eicu_prop_48$prop_adp_48<=0.5 & criteria ,]

### 48h mimic

In [350]:
df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.1 & df_mimic_prop_48$prop_adp_48<=0.3 & criteria ,]
df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.3 & df_mimic_prop_48$prop_adp_48<=0.5 & criteria ,]

In [352]:
df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_asp_48>=0.1 & df_mimic_prop_48$prop_asp_48<=0.3 & criteria ,]
df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_asp_48>=0.3 & df_mimic_prop_48$prop_asp_48<=0.5 & criteria ,]

In [354]:
df_10<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.1 & df_mimic_prop_48$prop_adp_48<=0.3 & criteria ,]
df_40<-df_mimic_prop_48[df_mimic_prop_48$prop_adp_48>=0.3 & df_mimic_prop_48$prop_adp_48<=0.5 & criteria ,]

### 72h eicu

In [405]:
df_10<-df_eicu_prop_72[df_eicu_prop_72$prop_map_72>=0.1 & df_eicu_prop_72$prop_map_72<=0.3 & criteria ,]
df_40<-df_eicu_prop_72[df_eicu_prop_72$prop_map_72>=0.3 & df_eicu_prop_72$prop_map_72<=0.5 & criteria ,]

In [407]:
df_10<-df_eicu_prop_72[df_eicu_prop_72$prop_asp_72>=0.1 & df_eicu_prop_72$prop_asp_72<=0.3 & criteria ,]
df_40<-df_eicu_prop_72[df_eicu_prop_72$prop_asp_72>=0.3 & df_eicu_prop_72$prop_asp_72<=0.5 & criteria ,]

In [409]:
df_10<-df_eicu_prop_72[df_eicu_prop_72$prop_adp_72>=0.1 & df_eicu_prop_72$prop_adp_72<=0.3 & criteria ,]
df_40<-df_eicu_prop_72[df_eicu_prop_72$prop_adp_72>=0.3 & df_eicu_prop_72$prop_adp_72<=0.5 & criteria ,]

### 72h mimic

In [356]:
df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.1 & df_mimic_prop_72$prop_adp_72<=0.3 & criteria ,]
df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.3 & df_mimic_prop_72$prop_adp_72<=0.5 & criteria ,]

In [358]:
df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.1 & df_mimic_prop_72$prop_asp_72<=0.3 & criteria ,]
df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_asp_72>=0.5 & df_mimic_prop_72$prop_asp_72<=0.5 & criteria ,]

In [360]:
df_10<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.1 & df_mimic_prop_72$prop_adp_72<=0.3 & criteria ,]
df_40<-df_mimic_prop_72[df_mimic_prop_72$prop_adp_72>=0.3 & df_mimic_prop_72$prop_adp_72<=0.5 & criteria ,]

# AKI ORs and CI

In [94]:
head(df_eicu_prop_AKI, n = 2)

patientunitstayid,age,gender,ethnicity,unittype,icu_mortality,hospital_expire_flag,apache_iv,sofatotal,vasso_duration_24h,⋯,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24,prop_map_24_AKI,prop_asp_24_AKI,prop_adp_24_AKI
141194,68,M,Caucasian,CTICU,0,0,70,3,0.00,⋯,0.04166667,0.7500000,0.04166667,0.2083333,1.0000000,0.000000000,0.00000000,0.04166667,0.2500000,0.00000000
141233,81,F,Caucasian,CTICU,0,0,66,12,23.97,⋯,0.50467290,0.3271028,0.45794393,0.2149533,0.9065421,0.009345794,0.08411215,0.65420561,0.6728972,0.09345794


In [614]:
criteria <-df_mimic_prop$ethnicity=="HISPANIC"

# 24h eicu

In [615]:
df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.05 & df_eicu_prop_AKI$prop_map_24_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.35 & df_eicu_prop_AKI$prop_map_24_AKI<=0.45 & criteria ,]

Warning message in df_eicu_prop_AKI$prop_map_24_AKI >= 0.05 & df_eicu_prop_AKI$prop_map_24_AKI <= :
“longer object length is not a multiple of shorter object length”Warning message in df_eicu_prop_AKI$prop_map_24_AKI >= 0.35 & df_eicu_prop_AKI$prop_map_24_AKI <= :
“longer object length is not a multiple of shorter object length”

In [97]:
df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.05 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.35 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.45 & criteria ,]

In [99]:
df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.05 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.35 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.45 & criteria ,]

In [827]:
criteria <-df_mimic_prop_AKI$cld==1

# 24h mimic

In [828]:
df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_24_AKI>=0.1 & df_mimic_prop_AKI$prop_map_24_AKI<=0.5 & criteria ,]
df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_map_24_AKI>=0.5 & df_mimic_prop_AKI$prop_map_24_AKI<=0.9 & criteria ,]

In [832]:
df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_asp_24_AKI>=0.05 & df_mimic_prop_AKI$prop_asp_24_AKI<=0.15 & criteria ,]
df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_asp_24_AKI>=0.35 & df_mimic_prop_AKI$prop_asp_24_AKI<=0.45 & criteria ,]

In [811]:
df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_adp_24_AKI>=0.001 & df_mimic_prop_AKI$prop_adp_24_AKI<=0.5 & criteria ,]
df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_adp_24_AKI>=0.5 & df_mimic_prop_AKI$prop_adp_24_AKI<=0.9 & criteria ,]

# 48h eicu

In [111]:
df_10<-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$prop_map_48_AKI>=0.05 & df_eicu_prop_AKI_48$prop_map_48_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$prop_map_48_AKI>=0.35 & df_eicu_prop_AKI_48$prop_map_48_AKI<=0.45 & criteria ,]

In [113]:
df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.05 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.35 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.45 & criteria ,]

In [115]:
df_10<-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$prop_adp_48_AKI>=0.05 & df_eicu_prop_AKI_48$prop_adp_48_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI_48[df_eicu_prop_AKI_48$prop_adp_48_AKI>=0.35 & df_eicu_prop_AKI_48$prop_adp_48_AKI<=0.45 & criteria ,]

# 48h mimic

In [814]:
df_10<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_map_48_AKI>=0.001 & df_mimic_prop_AKI_48$prop_map_48_AKI<=0.4 & criteria ,]
df_40<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_map_48_AKI>=0.4 & df_mimic_prop_AKI_48$prop_map_48_AKI<=0.6 & criteria ,]

In [816]:
df_10<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_asp_48_AKI>=0.001 & df_mimic_prop_AKI_48$prop_asp_48_AKI<=0.4 & criteria ,]
df_40<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_asp_48_AKI>=0.4 & df_mimic_prop_AKI_48$prop_asp_48_AKI<=0.6 & criteria ,]

In [818]:
df_10<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_adp_48_AKI>=0.001 & df_mimic_prop_AKI_48$prop_adp_48_AKI<=0.4 & criteria ,]
df_40<-df_mimic_prop_AKI_48[df_mimic_prop_AKI_48$prop_adp_48_AKI>=0.4 & df_mimic_prop_AKI_48$prop_adp_48_AKI<=0.6 & criteria ,]

# 72h eicu

In [123]:
df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.45 & criteria ,]

In [125]:
df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.45 & criteria ,]

In [127]:
df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.15 & criteria ,]
df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.45 & criteria ,]

# 72h mimic

In [821]:
df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_map_72_AKI>=0.001 & df_mimic_prop_AKI_72$prop_map_72_AKI<=0.5 & criteria ,]
df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_map_72_AKI>=0.5 & df_mimic_prop_AKI_72$prop_map_72_AKI<=0.9 & criteria ,]

In [823]:
df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_asp_72_AKI>=0.001 & df_mimic_prop_AKI_72$prop_asp_72_AKI<=0.4 & criteria ,]
df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_asp_72_AKI>=0.4 & df_mimic_prop_AKI_72$prop_asp_72_AKI<=0.6 & criteria ,]

In [825]:
df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_adp_72_AKI>=0.001 & df_mimic_prop_AKI_72$prop_adp_72_AKI<=0.4 & criteria ,]
df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_adp_72_AKI>=0.4 & df_mimic_prop_AKI_72$prop_adp_72_AKI<=0.6 & criteria ,]

In [826]:
a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
c<-length(df_10[df_10$AKI_7d==0,1])
b<-length(df_40[df_40$AKI_7d==1,1]) # control
d<-length(df_40[df_40$AKI_7d==0,1])
print(c(a,b,c,d))
a/(a+c)
b/(b+d)
oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

[1] 369 394  29  44


[1] 0.9271357

[1] 0.8995434

,Disease1,Disease2,Total
Exposed1,369,29,398
Exposed2,394,44,438
Total,763,73,836
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.417875,0.8717049,2.338943
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.1604129,0.1777866,0.1581307


## ORs and CI for subgroups

In [202]:
head(eicu_prop) 

X,patientunitstayid,propBelow_map_24,propAbove_map_24,prop_map_24,propBelow_asp_24,propAbove_asp_24,prop_asp_24,propBelow_adp_24,propAbove_adp_24,prop_adp_24
0,141168,0.1000000,0.3000000,0.60000000,NA,NA,NA,NA,NA,NA
1,141194,0.9583333,0.0000000,0.04166667,0.75000000,0.04166667,0.2083333,1.0000000,0.000000000,0.00000000
2,141203,0.3626374,0.3186813,0.31868132,NA,NA,NA,NA,NA,NA
3,141229,0.0000000,0.7500000,0.25000000,NA,NA,NA,NA,NA,NA
4,141233,0.3457944,0.1495327,0.50467290,0.32710280,0.45794393,0.2149533,0.9065421,0.009345794,0.08411215
5,141244,0.1052632,0.7368421,0.15789474,0.05263158,0.78947368,0.1578947,0.1052632,0.421052632,0.47368421


In [749]:
df_OR_CI

OR,LCI,UCI,pvalue
0.8351778,0.09173041,24.83445,1
0.8130954,0.08933385,24.17250,1


In [770]:
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.001 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.5 & df_eicu_prop$ckd==1,]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.5 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.9 & df_eicu_prop$ckd==1,]

    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))


,Disease1,Disease2,Total
Exposed1,324,38,362
Exposed2,408,49,457
Total,732,87,819
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.023068,0.6540786,1.611262
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.9206817,1,0.9173925


In [767]:
c

[1] 0

In [1020]:
c1 <- df_eicu_prop_48$ethnicity=="Asian"


# Criterias <- data.frame("c8"=c8,"c9"=c9,"c10"=c10 )
Criterias <- data.frame("c1"=c1)
# Criterias <- data.frame("c1"=c1,"c2"=c2,"C3" = c3, "c4"=c4, "C5" = c5 , "c6"=c6,"c7"=c7, "c8"=c8,"c9"=c9,"c10"=c10)
# Criterias <- data.frame("c6"=c10)

Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_map_24>=0.001 & df_eicu_prop_48$prop_map_24<=0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_map_24>=0.50 & df_eicu_prop_48$prop_map_24<=0.90 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_asp_24>=0.001 & df_eicu_prop_48$prop_asp_24<=0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_asp_24>=0.50 & df_eicu_prop_48$prop_asp_24<=0.90 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_adp_24>=0.001 & df_eicu_prop_48$prop_adp_24<=0.50 & Criterias[,i],]
    df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_adp_24>=0.50 & df_eicu_prop_48$prop_adp_24<=0.90 & Criterias[,i],]
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]

    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+3
    
}

ERROR: Error in uniroot(function(or) {: f() values at end points not of opposite sign


OR,LCI,UCI,pvalue
1.4399475,0.52913735,4.697192,0.6191871
0.3825042,0.05522859,3.442085,0.2884536
1.0588918,0.44095628,2.699490,1.0000000


In [2222]:
c1 <- df_eicu_prop_AKI$afib==1

In [2227]:
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.001 & df_eicu_prop_AKI$prop_map_24_AKI<=0.5 & c1,]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.5 & df_eicu_prop_AKI$prop_map_24_AKI<=0.9 & c1,]

In [2229]:
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.1 & df_eicu_prop_AKI$prop_map_24_AKI<=0.3 & c1,]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_map_24_AKI>=0.3 & df_eicu_prop_AKI$prop_map_24_AKI<=0.5 & c1,]

In [2220]:
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.1 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.5 & c1,]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_asp_24_AKI>=0.5 & df_eicu_prop_AKI$prop_asp_24_AKI<=0.9 & c1,]

In [2217]:
    df_10<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.1 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.5 & c1,]
    df_40<-df_eicu_prop_AKI[df_eicu_prop_AKI$prop_adp_24_AKI>=0.5 & df_eicu_prop_AKI$prop_adp_24_AKI<=0.9 & c1,]

In [2230]:
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

,Disease1,Disease2,Total
Exposed1,102,15,117
Exposed2,217,22,239
Total,319,37,356
,estimate,lower,upper
Exposed1,1.0000000,NA,NA
Exposed2,0.6881189,0.3434238,1.412978
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.3020652,0.3551527,0.2937195


In [1465]:
    df_10<-df_eicu_prop_48[df_eicu_prop_48$prop_map_48>=0.001 & df_eicu_prop_48$prop_map_48<=0.5 & c1,]
    df_40<-df_eicu_prop_48[df_eicu_prop_48$prop_map_48>=0.5   & df_eicu_prop_48$prop_map_48<=0.9 & c1,]

In [33]:
    df_10<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_adp_24_AKI>=0.001 & df_mimic_prop_AKI$prop_adp_24_AKI<=0.2 ,]
    df_40<-df_mimic_prop_AKI[df_mimic_prop_AKI$prop_adp_24_AKI>=0.3 & df_mimic_prop_AKI$prop_adp_24_AKI<=0.5 ,]

In [1866]:
    df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_map_72_AKI>=0.1 & df_mimic_prop_AKI_72$prop_map_72_AKI<=0.5 & c1,]
    df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_map_72_AKI>=0.5 & df_mimic_prop_AKI_72$prop_map_72_AKI<=0.9 & c1,]

In [1868]:
    df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_asp_72_AKI>=0.1 & df_mimic_prop_AKI_72$prop_asp_72_AKI<=0.5 & c1,]
    df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_asp_72_AKI>=0.5 & df_mimic_prop_AKI_72$prop_asp_72_AKI<=0.9 & c1,]

In [1870]:
    df_10<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_adp_72_AKI>=0.1 & df_mimic_prop_AKI_72$prop_adp_72_AKI<=0.5 & c1,]
    df_40<-df_mimic_prop_AKI_72[df_mimic_prop_AKI_72$prop_adp_72_AKI>=0.5 & df_mimic_prop_AKI_72$prop_adp_72_AKI<=0.9 & c1,]

In [34]:
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

,Disease1,Disease2,Total
Exposed1,302,40,342
Exposed2,783,145,928
Total,1085,185,1270
,estimate,lower,upper
Exposed1,1.00000,NA,NA
Exposed2,1.39423,0.9670279,2.05112
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.07564978,0.0882742,0.07829251


In [1463]:
    a<-length(df_10[df_10$hospital_expire_flag==1,1]) # exposed
    c<-length(df_10[df_10$hospital_expire_flag==0,1])
    b<-length(df_40[df_40$hospital_expire_flag==1,1]) # control
    d<-length(df_40[df_40$hospital_expire_flag==0,1])
    oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))

,Disease1,Disease2,Total
Exposed1,220,656,876
Exposed2,38,132,170
Total,258,788,1046
,estimate,lower,upper
Exposed1,1.000000,NA,NA
Exposed2,1.161835,0.7917288,1.73948
,midp.exact,fisher.exact,chi.square
Exposed1,NA,NA,NA
Exposed2,0.4496706,0.4965104,0.4446826


In [1691]:
c01 <- df_eicu_prop_AKI_72$vasso_duration_72h>0
c02 <- df_eicu_prop_AKI_72$vasso_duration_72h==0
c03 <- df_eicu_prop_AKI_72$vasso_duration_72h>=36
c04 <- df_eicu_prop_AKI_72$vasso_duration_72h<36
c1 <- df_eicu_prop_AKI_72$sepsis==1
c2 <- df_eicu_prop_AKI_72$sepsis==0
c3 <- df_eicu_prop_AKI_72$age>=65
c4 <- df_eicu_prop_AKI_72$age<=65
c5 <- df_eicu_prop_AKI_72$hypertension==1
c6<-  df_eicu_prop_AKI_72$hypertension==0
c7<-  df_eicu_prop_AKI_72$mechanical_ventilation==1
c8<-  df_eicu_prop_AKI_72$mechanical_ventilation==0
c9<- df_eicu_prop_AKI_72$bmi>=30
c10<- df_eicu_prop_AKI_72$bmi<30
# Criterias <- data.frame("c1"=c1,"c2"=c2,"C3" = c3, "c4"=c4, "C5" = c5 , "c6"=c6,"c7"=c7, "c8"=c8,"c9"=c9,
#                         "c10"=c10,"c11"=c11, "c12"=c12,"c13"=c13)

Criterias <- data.frame( c01, c02,c03, c04, c1, c2, "C3" = c3, "c4"=c4, "C5" = c5 , "c6"=c6,"c7"=c7, "c8"=c8,"c9"=c9,
                        "c10"=c10)

Hosp_OR<- list()
Hosp_LCI<-list()
Hosp_UCI<-list()
Hosp_pvalue<-list()
i=1
j=1
while (i <= length(Criterias)){
    # MAP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_map_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_map_72_AKI<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j]<-df_OR$measure[2]
    Hosp_LCI[j]<-df_OR$measure[4]
    Hosp_UCI[j]<-df_OR$measure[6]
    Hosp_pvalue[j]<-df_OR$p.value[4]

    # SBP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_asp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_asp_72_AKI<=0.45 & Criterias[,i],]

    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+1]<-df_OR$measure[2]
    Hosp_LCI[j+1]<-df_OR$measure[4]
    Hosp_UCI[j+1]<-df_OR$measure[6]
    Hosp_pvalue[j+1]<-df_OR$p.value[4]

    # DBP 24
    df_10<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.05 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.15 & Criterias[,i],]
    df_40<-df_eicu_prop_AKI_72[df_eicu_prop_AKI_72$prop_adp_72_AKI>=0.35 & df_eicu_prop_AKI_72$prop_adp_72_AKI<=0.45 & Criterias[,i],]
    a<-length(df_10[df_10$AKI_7d==1,1]) # exposed
    c<-length(df_10[df_10$AKI_7d==0,1])
    b<-length(df_40[df_40$AKI_7d==1,1]) # control
    d<-length(df_40[df_40$AKI_7d==0,1])
    df_OR<-oddsratio(matrix(c(a,b,c,d), ncol=2, nrow=2))
    Hosp_OR[j+2]<-df_OR$measure[2]
    Hosp_LCI[j+2]<-df_OR$measure[4]
    Hosp_UCI[j+2]<-df_OR$measure[6]
    Hosp_pvalue[j+2]<-df_OR$p.value[4]
 
    df_OR_CI <- data.frame("OR" = unlist(Hosp_OR, use.names=FALSE), 
                        "LCI" = unlist(Hosp_LCI, use.names=FALSE)
                       , "UCI" = unlist(Hosp_UCI, use.names=FALSE)
                       , "pvalue" = unlist(Hosp_pvalue, use.names=FALSE))
    i <-i+1
    j <-j+9
    
}
df_OR_CI

Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”Warning message in chisq.test(xx, correct = correction):
“Chi-squared approximation may be incorrect”

OR,LCI,UCI,pvalue
1.8443556,0.86270028,4.603601,0.16192083
1.3869890,0.86746617,2.284484,0.20123062
1.4789430,0.75512425,3.084752,0.31890240
0.6285004,0.26216270,1.686166,0.31182383
1.5258745,0.83466610,2.962230,0.18846232
0.6453537,0.31598887,1.355086,0.25077920
2.2936189,0.88305574,8.007393,0.14413957
2.1054648,1.13157293,4.238108,0.02254494
1.6570452,0.73450458,4.124382,0.31917104
0.7794125,0.38016903,1.735398,0.54688022
